# スタッキング001

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Lgb, Model1Xgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



In [3]:
path = './home-credit-default-risk/'

## 共通関数定義

In [4]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [5]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [6]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [7]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成1

In [8]:
### bureau_balance
bureau = FeatureEngineering.bureau_and_balance(path)
bureau.head()

Memory usage of dataframe: 4114.69 MB
Memory usage after optimization: 1189.22 MB
Decreased by 71.1%


,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [9]:
bureau = bureau.replace([np.inf, -np.inf], np.nan)
###bureau = bureau.fillna(bureau.max() + (bureau.max() * 0.4))
bureau.head()

,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [10]:
### previos_application
previous = FeatureEngineering.previous_application(path)
previous.head()

Memory usage of dataframe: 2434.34 MB
Memory usage after optimization: 818.55 MB
Decreased by 66.4%


,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [11]:
previous = previous.replace([np.inf, -np.inf], np.nan)
###previous = previous.fillna(previous.max() + (previous.max() * 0.4))
previous.head()

,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [12]:
### pos_cash
pos = FeatureEngineering.pos_cash(path)
pos.head()

Memory usage of dataframe: 147.95 MB
Memory usage after optimization: 46.96 MB
Decreased by 68.3%


,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [13]:
pos = pos.replace([np.inf, -np.inf], np.nan)
###pos = pos.fillna(pos.max() + (pos.max() * 0.4))
pos.head()

,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [14]:
### instllment_payment
installments = FeatureEngineering.installments_payments(path)
installments.head()

Memory usage of dataframe: 199.49 MB
Memory usage after optimization: 73.84 MB
Decreased by 63.0%


,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [15]:
installments = installments.replace([np.inf, -np.inf], np.nan)
###installments = installments.fillna(installments.max() + (installments.max() * 0.4))
installments.head()

,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [16]:
### credit_card_balance
credit = FeatureEngineering.credit_card_balance(path)
credit.head()

Memory usage of dataframe: 173.03 MB
Memory usage after optimization: 66.37 MB
Decreased by 61.6%


,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [17]:
credit = credit.replace([np.inf, -np.inf], np.nan)
###credit = credit.fillna(credit.max() + (credit.max() * 0.4))
credit.head()

,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [18]:
### application
df = FeatureEngineering.application_train_test(path, 'application_train.csv')
df.head()

Memory usage of dataframe: 404.30 MB
Memory usage after optimization: 169.87 MB
Decreased by 58.0%


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,app_AMT_INCOME_TOTAL_div_CNT_CHILDREN,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,202500.0,0,0,NaN,NaN,0.119873,1.780273,8824.0,0.067322,0.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,270000.0,0,0,NaN,NaN,0.049377,0.696777,15576.0,0.070862,0.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,67500.0,0,0,-0.001365,-0.11554,0.042786,3.623047,18816.0,0.011810,0.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,135000.0,0,0,NaN,NaN,0.032471,0.203003,15968.0,0.159912,0.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,121500.0,0,0,NaN,NaN,0.055481,0.364014,16896.0,0.152466,0.0


In [19]:
df = df.replace([np.inf, -np.inf], np.nan)
#df = df.fillna(installments.max() + (installments.max() * 0.4))
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,app_AMT_INCOME_TOTAL_div_CNT_CHILDREN,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,202500.0,0,0,NaN,NaN,0.119873,1.780273,8824.0,0.067322,0.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,270000.0,0,0,NaN,NaN,0.049377,0.696777,15576.0,0.070862,0.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,67500.0,0,0,-0.001365,-0.11554,0.042786,3.623047,18816.0,0.011810,0.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,135000.0,0,0,NaN,NaN,0.032471,0.203003,15968.0,0.159912,0.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,121500.0,0,0,NaN,NaN,0.055481,0.364014,16896.0,0.152466,0.0


### データフレームを結合

In [20]:
### データフレームをマージ
df = pd.merge(df, bureau, on='SK_ID_CURR', how='left')
df = pd.merge(df, installments, on='SK_ID_CURR', how='left')
df = pd.merge(df, previous, on='SK_ID_CURR', how='left')
df = pd.merge(df, credit, on='SK_ID_CURR', how='left')
df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


In [21]:
#df = FeatureEngineering.clean_data(df)
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


In [22]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 3396 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 3279 features are remained after removing non-informative features
1142 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 2137 features are remained after removing features not interesting for LightGBM classifier
---=> 90 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 2227 features

---=> df final shape: (356249, 2227)  <=--- 



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,1,0,1,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 2227 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 2223 features are remained after removing non-informative features
111 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 2112 features are remained after removing features not interesting for LightGBM classifier
---=> 31 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 2143 features

---=> df final shape: (356249, 2143)  <=--- 



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,APPROVED_NAME_GOODS_CATEGORY_Insurance_MEAN_high_risk,APPROVED_NAME_GOODS_CATEGORY_Insurance_MEAN_low_risk,REFUSED_prev_missing_MIN_high_risk,REFUSED_prev_missing_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1,0,1,0,0,1,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,1,0,0,1,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,0,1,0,0,1,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,1,0,1,0,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,0,1,0,0,1,0,0,0,0


In [24]:
### 変数選択003
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 2143 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 2141 features are remained after removing non-informative features
53 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 2088 features are remained after removing features not interesting for LightGBM classifier
---=> 24 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 2112 features

---=> df final shape: (356249, 2112)  <=--- 



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,ACTIVE_Last_status_0_MEAN_SUM_low_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,1,0,1,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,1,0,1,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,1,0,1,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,0,1,0,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,1,0,1,0,0,0,0,0,0


In [25]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,ACTIVE_Last_status_0_MEAN_SUM_low_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,1,0,1,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,1,0,1,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,1,0,1,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,0,1,0,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,1,0,1,0,0,0,0,0,0


In [26]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)
df.head()

(356249, 2112)


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,ACTIVE_Last_status_0_MEAN_SUM_low_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,1,0,1,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,1,0,1,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,1,0,1,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,0,1,0,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,1,0,1,0,0,0,0,0,0


In [27]:
### split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 2112)


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,ACTIVE_Last_status_0_MEAN_SUM_low_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
307505,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,1,0,1,0,0,0,0,0,0
307506,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,1,0,0,0,0,0,0
307507,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,1,0,1,0,1,0
307508,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,0,1,1,0,1,0,1,0
307509,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,1,0,0,0,0,0,0


In [28]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [29]:
app_train = app_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,1,0,0,0,0,0,0,0.051055
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,0,1,0,0,0,0,0,0,0.166067
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,1,0,1,0,1,0,0.043899
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,1,0,1,0,1,0,0.056542
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,1,0,0,0,0,0,0,0.183168


In [30]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.025023
1,100005,0.090335
2,100013,0.037033
3,100028,0.036644
4,100038,0.172674


In [31]:
app_train = app_train.merge(train_set2a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2a, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,1,0,0,0,0,0,0,0.051055,0.025023
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,0,0,0,0,0,0.166067,0.090335
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,1,1,0,1,0,1,0,0.043899,0.037033
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,1,0,1,0,1,0,0.056542,0.036644
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,0,0,0,0,0,0.183168,0.172674


In [32]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.038205
1,100005,0.103087
2,100013,0.058075
3,100028,0.035603
4,100038,0.184927


In [33]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,0,0,0,0,0,0.051055,0.025023,0.038205
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,0,0,0,0,0,0,0.166067,0.090335,0.103087
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.043899,0.037033,0.058075
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.056542,0.036644,0.035603
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,0,0,0,0,0,0.183168,0.172674,0.184927


In [34]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,0,0,0,0,0.051055,0.025023,0.038205
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,0,0,0,0,0.166067,0.090335,0.103087
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.043899,0.037033,0.058075
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.056542,0.036644,0.035603
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,1,0,0,0,0,0,0,0.183168,0.172674,0.184927


In [35]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,0,0,0,0,0,0.051055,0.025023,0.038205
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,0,0,0,0,0,0,0.166067,0.090335,0.103087
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.043899,0.037033,0.058075
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.056542,0.036644,0.035603
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,0,0,0,0,0,0.183168,0.172674,0.184927


In [36]:
# infをnanへ置換
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,0,0,0,0,0,0.051055,0.025023,0.038205
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,0,0,0,0,0,0,0.166067,0.090335,0.103087
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.043899,0.037033,0.058075
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.056542,0.036644,0.035603
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,0,0,0,0,0,0.183168,0.172674,0.184927


### modelを生成

In [37]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [38]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [39]:
# infをnanへ置換
#app_train = app_train.fillna(app_train.max() + (app_train.max() * 0.4))
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1,0,0,0,0,0,0,0.516856,0.156048,0.254294
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,0,0,0,0,0,0.010645,0.012034,0.022026
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,0,0,0,0,0,0,0.054077,0.026333,0.035999
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,0,1,0,1,0.059492,0.029856,0.021632
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,0,0,0,0,0,0,0.076165,0.034698,0.025333


In [40]:
# 欠損値補完
app_test = app_test.replace([np.inf, -np.inf], np.nan)
#app_test = app_test.fillna(app_test.max() + (app_test.max() * 0.4))
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,0,0,0,0,0,0.051055,0.025023,0.038205
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,0,0,0,0,0,0,0.166067,0.090335,0.103087
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.043899,0.037033,0.058075
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.056542,0.036644,0.035603
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,0,0,0,0,0,0.183168,0.172674,0.184927


In [41]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [42]:
### indexを設定
app_train = app_train.set_index('SK_ID_CURR')
app_test = app_test.set_index('SK_ID_CURR')
app_test.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018845,...,1,0,0,0,0,0,0,0.051055,0.025023,0.038205
100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035797,...,1,0,0,0,0,0,0,0.166067,0.090335,0.103087
100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019104,...,1,1,0,1,0,1,0,0.043899,0.037033,0.058075
100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,...,1,1,0,1,0,1,0,0.056542,0.036644,0.035603
100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010033,...,1,0,0,0,0,0,0,0.183168,0.172674,0.184927


In [43]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,0.720375,-0.717914,-0.664541,-0.577536,-0.307107,-0.075103,-0.451788,-0.239271,-0.146484,...,0.769932,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,-0.369431,-0.522813,-0.436778
100005,NaN,-1.388165,-0.717914,-0.664541,-0.577536,-0.637955,-0.934824,-0.671917,-0.970093,1.079102,...,0.769932,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,0.969466,0.166746,0.236692
100013,NaN,-1.388165,1.392925,-0.664541,-0.577536,0.313233,0.159594,2.943901,0.247945,-0.127563,...,0.769932,1.685051,-0.150514,2.007797,-0.302406,2.007367,-0.302336,-0.452740,-0.396011,-0.230535
100028,NaN,0.720375,-0.717914,-0.664541,2.192089,1.347132,2.424815,1.511670,2.805824,0.399658,...,0.769932,1.685051,-0.150514,2.007797,-0.302406,2.007367,-0.302336,-0.305557,-0.400115,-0.463788
100038,NaN,-1.388165,1.392925,1.504799,0.807276,0.106453,0.065769,0.342102,0.235764,-0.783203,...,0.769932,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,1.168553,1.036070,1.086179


In [44]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,100001,NaN,0.720375,-0.717914,-0.664541,-0.577536,-0.307107,-0.075103,-0.451788,-0.239271,...,0.769932,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,-0.369431,-0.522813,-0.436778
1,100005,NaN,-1.388165,-0.717914,-0.664541,-0.577536,-0.637955,-0.934824,-0.671917,-0.970093,...,0.769932,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,0.969466,0.166746,0.236692
2,100013,NaN,-1.388165,1.392925,-0.664541,-0.577536,0.313233,0.159594,2.943901,0.247945,...,0.769932,1.685051,-0.150514,2.007797,-0.302406,2.007367,-0.302336,-0.452740,-0.396011,-0.230535
3,100028,NaN,0.720375,-0.717914,-0.664541,2.192089,1.347132,2.424815,1.511670,2.805824,...,0.769932,1.685051,-0.150514,2.007797,-0.302406,2.007367,-0.302336,-0.305557,-0.400115,-0.463788
4,100038,NaN,-1.388165,1.392925,1.504799,0.807276,0.106453,0.065769,0.342102,0.235764,...,0.769932,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,1.168553,1.036070,1.086179


In [45]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,100002,1.0,-1.387695,-0.717773,-0.664551,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,...,0.770020,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302246,5.054688,0.860352,1.806641
1,100003,0.0,0.720215,-0.717773,1.504883,-0.577637,0.933594,1.725586,0.592773,1.599609,...,0.770020,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302246,-0.839844,-0.660156,-0.604492
2,100004,0.0,-1.387695,1.392578,-0.664551,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,...,0.770020,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302246,-0.334229,-0.508789,-0.459717
3,100006,0.0,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,...,-1.298828,1.684570,-0.150513,-0.498047,3.306641,-0.498047,3.306641,-0.271240,-0.471680,-0.608887
4,100007,0.0,-1.387695,-0.717773,-0.664551,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,...,0.770020,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302246,-0.077087,-0.420654,-0.570312


In [46]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,pred_1b,set2a,set3a
0,100001,NaN,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.075073,-0.451904,-0.239258,...,0.77002,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.302246,-0.369385,-0.522949,-0.436768
1,100005,NaN,-1.387695,-0.717773,-0.664551,-0.577637,-0.638184,-0.935059,-0.671875,-0.970215,...,0.77002,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.302246,0.969238,0.166748,0.236694
2,100013,NaN,-1.387695,1.392578,-0.664551,-0.577637,0.313232,0.159546,2.943359,0.247925,...,0.77002,1.684570,-0.150513,2.007812,-0.30249,2.007812,-0.302246,-0.452637,-0.395996,-0.230591
3,100028,NaN,0.720215,-0.717773,-0.664551,2.191406,1.346680,2.425781,1.511719,2.806641,...,0.77002,1.684570,-0.150513,2.007812,-0.30249,2.007812,-0.302246,-0.305664,-0.400146,-0.463867
4,100038,NaN,-1.387695,1.392578,1.504883,0.807129,0.106445,0.065796,0.342041,0.235718,...,0.77002,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.302246,1.168945,1.036133,1.085938


In [47]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [48]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [49]:
### training Xgb
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Lgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, x_train, y_train, x_test)

[LightGBM] [Info] Number of positive: 18660, number of negative: 211968
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.168618 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 369873
[LightGBM] [Info] Number of data points in the train set: 230628, number of used features: 2110
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080910 -> initscore=-2.430053
[LightGBM] [Info] Start training from score -2.430053
[1]	train's auc: 0.669728	valid's auc: 0.663241
[2]	train's auc: 0.69303	valid's auc: 0.683499
[3]	train's auc: 0.70795	valid's auc: 0.695743
[4]	train's auc: 0.72743	valid's auc: 0.716258
[5]	train's auc: 0.740671	valid's auc: 0.730838
[6]	train's auc: 0.760371	valid's auc: 0.7552
[7]	train's auc: 0.761517	valid's auc: 0.755743
[8]	train's auc: 0.777714	valid's auc: 0.773118
[9]	train's auc: 0.777084	valid's auc: 0.771713
[10]	t

[158]	train's auc: 0.796133	valid's auc: 0.785544
[159]	train's auc: 0.796233	valid's auc: 0.785565
[160]	train's auc: 0.796292	valid's auc: 0.785594
[161]	train's auc: 0.79636	valid's auc: 0.785698
[162]	train's auc: 0.796428	valid's auc: 0.785711
[163]	train's auc: 0.796492	valid's auc: 0.785821
[164]	train's auc: 0.796618	valid's auc: 0.785912
[165]	train's auc: 0.796705	valid's auc: 0.785949
[166]	train's auc: 0.796764	valid's auc: 0.78595
[167]	train's auc: 0.796825	valid's auc: 0.785961
[168]	train's auc: 0.796872	valid's auc: 0.785962
[169]	train's auc: 0.796961	valid's auc: 0.786036
[170]	train's auc: 0.796992	valid's auc: 0.786059
[171]	train's auc: 0.797249	valid's auc: 0.786404
[172]	train's auc: 0.797315	valid's auc: 0.786442
[173]	train's auc: 0.797289	valid's auc: 0.786408
[174]	train's auc: 0.797384	valid's auc: 0.786438
[175]	train's auc: 0.797432	valid's auc: 0.786449
[176]	train's auc: 0.797454	valid's auc: 0.786428
[177]	train's auc: 0.797532	valid's auc: 0.786456
[1

[324]	train's auc: 0.807683	valid's auc: 0.792246
[325]	train's auc: 0.807745	valid's auc: 0.792263
[326]	train's auc: 0.807817	valid's auc: 0.792291
[327]	train's auc: 0.807884	valid's auc: 0.7923
[328]	train's auc: 0.807946	valid's auc: 0.792323
[329]	train's auc: 0.808004	valid's auc: 0.79234
[330]	train's auc: 0.808062	valid's auc: 0.792356
[331]	train's auc: 0.808133	valid's auc: 0.792389
[332]	train's auc: 0.808202	valid's auc: 0.79244
[333]	train's auc: 0.80825	valid's auc: 0.792455
[334]	train's auc: 0.808312	valid's auc: 0.792474
[335]	train's auc: 0.808408	valid's auc: 0.792554
[336]	train's auc: 0.808464	valid's auc: 0.79262
[337]	train's auc: 0.808505	valid's auc: 0.792635
[338]	train's auc: 0.808558	valid's auc: 0.792656
[339]	train's auc: 0.808625	valid's auc: 0.792746
[340]	train's auc: 0.808692	valid's auc: 0.792759
[341]	train's auc: 0.808747	valid's auc: 0.792768
[342]	train's auc: 0.808804	valid's auc: 0.792784
[343]	train's auc: 0.808865	valid's auc: 0.792793
[344]	

[490]	train's auc: 0.816454	valid's auc: 0.794968
[491]	train's auc: 0.816524	valid's auc: 0.794999
[492]	train's auc: 0.816573	valid's auc: 0.795013
[493]	train's auc: 0.816604	valid's auc: 0.795031
[494]	train's auc: 0.816654	valid's auc: 0.795026
[495]	train's auc: 0.816716	valid's auc: 0.795053
[496]	train's auc: 0.816766	valid's auc: 0.795057
[497]	train's auc: 0.816813	valid's auc: 0.795076
[498]	train's auc: 0.816864	valid's auc: 0.795077
[499]	train's auc: 0.816909	valid's auc: 0.795079
[500]	train's auc: 0.816957	valid's auc: 0.795094
[501]	train's auc: 0.817008	valid's auc: 0.795095
[502]	train's auc: 0.817063	valid's auc: 0.795093
[503]	train's auc: 0.81712	valid's auc: 0.795099
[504]	train's auc: 0.817168	valid's auc: 0.795092
[505]	train's auc: 0.81722	valid's auc: 0.795105
[506]	train's auc: 0.817239	valid's auc: 0.795125
[507]	train's auc: 0.817283	valid's auc: 0.795133
[508]	train's auc: 0.817332	valid's auc: 0.795135
[509]	train's auc: 0.817366	valid's auc: 0.795133
[5

[656]	train's auc: 0.823969	valid's auc: 0.796095
[657]	train's auc: 0.824011	valid's auc: 0.796094
[658]	train's auc: 0.824055	valid's auc: 0.796092
[659]	train's auc: 0.824101	valid's auc: 0.796093
[660]	train's auc: 0.824152	valid's auc: 0.796104
[661]	train's auc: 0.824193	valid's auc: 0.796104
[662]	train's auc: 0.82424	valid's auc: 0.796106
[663]	train's auc: 0.824287	valid's auc: 0.796118
[664]	train's auc: 0.824335	valid's auc: 0.796118
[665]	train's auc: 0.824381	valid's auc: 0.796121
[666]	train's auc: 0.824423	valid's auc: 0.796128
[667]	train's auc: 0.824467	valid's auc: 0.796119
[668]	train's auc: 0.824513	valid's auc: 0.796136
[669]	train's auc: 0.824556	valid's auc: 0.796145
[670]	train's auc: 0.824602	valid's auc: 0.796164
[671]	train's auc: 0.824649	valid's auc: 0.796161
[672]	train's auc: 0.824693	valid's auc: 0.796168
[673]	train's auc: 0.824739	valid's auc: 0.79617
[674]	train's auc: 0.824777	valid's auc: 0.796194
[675]	train's auc: 0.824818	valid's auc: 0.796196
[6

...

[1192]	train's auc: 0.844649	valid's auc: 0.793924
[1193]	train's auc: 0.844677	valid's auc: 0.793926
[1194]	train's auc: 0.844713	valid's auc: 0.793936
[1195]	train's auc: 0.84474	valid's auc: 0.793933
[1196]	train's auc: 0.844769	valid's auc: 0.793926
[1197]	train's auc: 0.844799	valid's auc: 0.793932
[1198]	train's auc: 0.844829	valid's auc: 0.793933
[1199]	train's auc: 0.844862	valid's auc: 0.793933
[1200]	train's auc: 0.844898	valid's auc: 0.793942
[1201]	train's auc: 0.844926	valid's auc: 0.79395
[1202]	train's auc: 0.844958	valid's auc: 0.793944
[1203]	train's auc: 0.844992	valid's auc: 0.793938
[1204]	train's auc: 0.845037	valid's auc: 0.793944
[1205]	train's auc: 0.845071	valid's auc: 0.793943
[1206]	train's auc: 0.845118	valid's auc: 0.793926
[1207]	train's auc: 0.845148	valid's auc: 0.793931
[1208]	train's auc: 0.845182	valid's auc: 0.793928
[1209]	train's auc: 0.845214	valid's auc: 0.793925
[1210]	train's auc: 0.845245	valid's auc: 0.793922
[1211]	train's auc: 0.845278	vali

[1354]	train's auc: 0.849815	valid's auc: 0.793858
[1355]	train's auc: 0.849854	valid's auc: 0.793845
[1356]	train's auc: 0.849895	valid's auc: 0.793846
[1357]	train's auc: 0.849925	valid's auc: 0.793843
[1358]	train's auc: 0.849957	valid's auc: 0.793844
[1359]	train's auc: 0.849984	valid's auc: 0.793839
[1360]	train's auc: 0.85003	valid's auc: 0.793832
[1361]	train's auc: 0.850057	valid's auc: 0.793837
[1362]	train's auc: 0.850088	valid's auc: 0.793833
[1363]	train's auc: 0.850113	valid's auc: 0.793834
[1364]	train's auc: 0.850138	valid's auc: 0.793832
[1365]	train's auc: 0.850166	valid's auc: 0.793829
[1366]	train's auc: 0.850193	valid's auc: 0.793832
[1367]	train's auc: 0.850225	valid's auc: 0.793828
[1368]	train's auc: 0.850247	valid's auc: 0.793821
[1369]	train's auc: 0.850281	valid's auc: 0.793821
[1370]	train's auc: 0.850306	valid's auc: 0.793814
[1371]	train's auc: 0.850335	valid's auc: 0.793809
[1372]	train's auc: 0.850373	valid's auc: 0.793819
[1373]	train's auc: 0.850406	val

[1516]	train's auc: 0.854761	valid's auc: 0.793714
[1517]	train's auc: 0.854791	valid's auc: 0.79371
[1518]	train's auc: 0.854819	valid's auc: 0.793706
[1519]	train's auc: 0.854843	valid's auc: 0.793709
[1520]	train's auc: 0.854869	valid's auc: 0.793705
[1521]	train's auc: 0.854902	valid's auc: 0.793704
[1522]	train's auc: 0.854931	valid's auc: 0.793712
[1523]	train's auc: 0.854957	valid's auc: 0.793712
[1524]	train's auc: 0.854984	valid's auc: 0.793702
[1525]	train's auc: 0.855007	valid's auc: 0.7937
[1526]	train's auc: 0.855036	valid's auc: 0.793703
[1527]	train's auc: 0.855066	valid's auc: 0.7937
[1528]	train's auc: 0.855094	valid's auc: 0.793694
[1529]	train's auc: 0.855117	valid's auc: 0.793691
[1530]	train's auc: 0.855145	valid's auc: 0.793687
[1531]	train's auc: 0.855179	valid's auc: 0.793687
[1532]	train's auc: 0.855202	valid's auc: 0.793683
[1533]	train's auc: 0.855227	valid's auc: 0.793688
[1534]	train's auc: 0.855245	valid's auc: 0.793688
[1535]	train's auc: 0.855289	valid's

[1678]	train's auc: 0.859334	valid's auc: 0.793385
[1679]	train's auc: 0.859367	valid's auc: 0.793377
[1680]	train's auc: 0.859399	valid's auc: 0.793372
[1681]	train's auc: 0.859424	valid's auc: 0.793375
[1682]	train's auc: 0.859449	valid's auc: 0.793382
[1683]	train's auc: 0.859477	valid's auc: 0.793381
[1684]	train's auc: 0.859508	valid's auc: 0.793384
[1685]	train's auc: 0.859532	valid's auc: 0.793386
[1686]	train's auc: 0.859557	valid's auc: 0.79339
[1687]	train's auc: 0.859586	valid's auc: 0.793384
[1688]	train's auc: 0.859615	valid's auc: 0.79339
[1689]	train's auc: 0.859644	valid's auc: 0.793392
[1690]	train's auc: 0.859672	valid's auc: 0.793397
[1691]	train's auc: 0.859686	valid's auc: 0.793393
[1692]	train's auc: 0.85973	valid's auc: 0.793383
[1693]	train's auc: 0.859753	valid's auc: 0.793374
[1694]	train's auc: 0.859782	valid's auc: 0.793369
[1695]	train's auc: 0.859804	valid's auc: 0.793376
[1696]	train's auc: 0.859827	valid's auc: 0.793376
[1697]	train's auc: 0.859852	valid

[1840]	train's auc: 0.863694	valid's auc: 0.793087
[1841]	train's auc: 0.86372	valid's auc: 0.793085
[1842]	train's auc: 0.863754	valid's auc: 0.79308
[1843]	train's auc: 0.863774	valid's auc: 0.793079
[1844]	train's auc: 0.863804	valid's auc: 0.793081
[1845]	train's auc: 0.86383	valid's auc: 0.793079
[1846]	train's auc: 0.863856	valid's auc: 0.793085
[1847]	train's auc: 0.863871	valid's auc: 0.793087
[1848]	train's auc: 0.863912	valid's auc: 0.79309
[1849]	train's auc: 0.86394	valid's auc: 0.793091
[1850]	train's auc: 0.86396	valid's auc: 0.793097
[1851]	train's auc: 0.863986	valid's auc: 0.793097
[1852]	train's auc: 0.864015	valid's auc: 0.793089
[1853]	train's auc: 0.864046	valid's auc: 0.793084
[1854]	train's auc: 0.864065	valid's auc: 0.793081
[1855]	train's auc: 0.864084	valid's auc: 0.793079
[1856]	train's auc: 0.864113	valid's auc: 0.793073
[1857]	train's auc: 0.864138	valid's auc: 0.793072
[1858]	train's auc: 0.864155	valid's auc: 0.793066
[1859]	train's auc: 0.864174	valid's 

In [50]:
pred_train_1a

array([0.26092759, 0.01217109, 0.03995003, ..., 0.02355114, 0.1180223 ,
       0.07925515])

In [51]:
pred_test_1a

array([0.04006768, 0.09524007, 0.04443514, ..., 0.00742228, 0.02078377,
       0.24974532])

In [52]:
pred_1a = []
for a in np.array(pred_train_1a):
    pred_1a.append(a)

len(pred_1a)

307505

In [53]:
print(type(pred_1a))

<class 'list'>


In [54]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set1': pred_1a})
print(train_x_2.shape)
train_x_2.head()

(307505, 2)


,SK_ID_CURR,set1
0,100002,0.260928
1,100003,0.012171
2,100004,0.039950
3,100006,0.030776
4,100007,0.035088


In [55]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/pred_train_set1a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [56]:
pred_1a = []
for a in np.array(pred_test_1a):
    pred_1a.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1a)

48744

In [57]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set1': pred_1a})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [58]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/pred_test_set1a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [59]:
#del df
#del x_train
#del y_train
#del x_test
#del app_train
#del app_test
#del model_1b
#gc.collect()

## 特徴量生成2